In [154]:
import os

data_dir = "gut/"

files = os.listdir(data_dir)
files = [x for x in files if x.endswith(".txt") and "__" in x]

In [186]:
texts = []
labels = []

for fn in files:
    with open(os.path.join(data_dir, fn), encoding="ISO-8859-1") as f:
        s = f.read()
    texts.append(s[1000:])
    labels.append(fn[:-4])

In [187]:
shortest = sorted([(len(text), i) for i, text in enumerate(texts)])[:36]

In [188]:
shortest_is = set([s[1] for s in shortest])

In [189]:
texts = [x for i, x in enumerate(texts) if i not in shortest_is]
labels = [x for i, x in enumerate(labels) if i not in shortest_is]

In [190]:
len(texts)

3000

In [191]:
def get_chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [192]:
title_chunks = {}  ## 'A Nonamous': ['chunk 1', 'chunk 5', 'chunk 2']

for i, text in enumerate(texts):
    chunks = get_chunks(text, 1000)
    num = 0
    for chunk in chunks:
        if num >= 50:
            break
        num += 1
        if labels[i] in title_chunks:
            title_chunks[labels[i]].append(chunk)
        else:
            title_chunks[labels[i]] = [chunk]
        

In [193]:
from random import randrange
import random

knowns = []
unknowns = []

for title in title_chunks:
    author = title.split("__")[0]
    same_authors = [t for t in title_chunks.keys() if author == t.split("__")[0] and t != title]
    diff_authors = [t for t in title_chunks.keys() if author != t.split("__")[0]]

    for chunk in title_chunks[title]:
        # try add a random chunk from a different title by the same author
        if len(same_authors) > 0:
            same_author = same_authors[random.randrange(len(same_authors))]
        # otherwise just add another chunk from this title
        else:
            same_author = title
        
        # same author text 
        sat = title_chunks[same_author][random.randrange(len(title_chunks[same_author]))]
        
        # different author text
        diff_author = diff_authors[random.randrange(len(diff_authors))]
        dat = title_chunks[diff_author][random.randrange(len(title_chunks[diff_author]))]
        
        knowns.append(chunk)
        knowns.append(chunk)
        unknowns.append(sat)
        unknowns.append(dat)

In [194]:
print(len(knowns))

labels = [1,0] * int(len(knowns) / 2)

285448


In [195]:
unknowns[0]

's Ginger, patting \'im on the\nweskit to make sure the locket was still there.  "It\'s a good job you\'ve\ngot us to look arter you."\n\n"We must buy \'im a money-belt with a pocket in it," ses Peter.\n\nGinger nodded at \'im.  "Yes," he ses, "that would be safer.  And he\'d\nbetter wear it next to \'is skin, with everything over it.  I should feel\nmore comfortable then."\n\n"And wot about me?" says Sam, turning on \'im.\n\n"Well, we\'ll take it in turns," ses Ginger.  "You one day, and then me,\nand then Peter."\n\nSam gave way at last, as arter all he could see it was the safest thing\nto do, but he \'ad so much to say about it that they got fair sick of the\nsound of \'is voice.  They \'ad to go \'ome for \'im to put the belt on; and\nthen at seven o\'clock in the evening, arter Sam had \'ad two or three\npints, they had to go \'ome agin, \'cos he was complaining of tight-lacing.\n\nGinger had it on next day and he went \'ome five times.  The other two\nwent with \'im in case he l

In [196]:
import numpy as np
from random import shuffle

indices = list(range(len(knowns)))
shuffle(indices)

In [197]:
knowns = np.array(knowns)
unknowns = np.array(unknowns)
labels = np.array(labels)

In [198]:
knowns = knowns[indices]
unknowns = unknowns[indices]
labels = labels[indices]

In [199]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [209]:
%%time
vectorizer = TfidfVectorizer(min_df=3, ngram_range=(1,2), lowercase=False)

vectorizer.fit(knowns)

CPU times: user 2min 22s, sys: 484 ms, total: 2min 22s
Wall time: 2min 22s


In [210]:
%%time
known_vecs = vectorizer.transform(knowns)

CPU times: user 2min 3s, sys: 12 ms, total: 2min 3s
Wall time: 2min 3s


In [211]:
%%time
unknown_vecs = vectorizer.transform(unknowns)

CPU times: user 2min 3s, sys: 12 ms, total: 2min 3s
Wall time: 2min 3s


In [212]:
diffs = np.abs(known_vecs - unknown_vecs)

In [213]:
labels[:10]

array([1, 1, 0, 1, 0, 0, 1, 1, 1, 0])

In [217]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, LinearSVC

nb = MultinomialNB(fit_prior=False, alpha=0.001)
svm = LinearSVC()

X_train, X_test, y_train, y_test = train_test_split(diffs, labels, test_size=0.5)

In [218]:
%%time
svm.fit(X_train, y_train)

CPU times: user 27.1 s, sys: 0 ns, total: 27.1 s
Wall time: 27.1 s


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [219]:
%%time
from sklearn.metrics import accuracy_score
preds = svm.predict(X_test)

print(accuracy_score(y_test, preds))

0.442020963538
CPU times: user 368 ms, sys: 0 ns, total: 368 ms
Wall time: 368 ms


In [208]:
Counter(preds)

Counter({0: 14285, 1: 14260})

In [ ]:
preds[:10]

In [ ]:
y_test[:10]

In [27]:
from sklearn.externals import joblib
joblib.dump(vectorizer,"/dev/data/vec.pickle")

['/dev/data/vec.pickle']

In [58]:
joblib.dump(nb,"/dev/data/nb_author.pickle")

['/dev/data/nb_author.pickle']

In [29]:
y_train[:10]

array(['James Fenimore Cooper___The Chainbearer',
       'George Alfred Henty___Held Fast For England',
       'Mark Twain___Tom Sawyer Abroad',
       'James Fenimore Cooper___The Deerslayer',
       "Robert Louis Stevenson___A Child's Garden of Verses, Verse 130",
       'Rudyard Kipling___Letters of Travel (1892-1913)',
       'Edgar Rice Burroughs___The People that Time Forgot',
       'George Alfred Henty___Through Russian Snows',
       'Bret Harte___Clarence', 'Andrew Lang___Adventures among Books'], 
      dtype='<U176')

In [30]:
y_train_authors = [y.split("__")[0] for y in y_train]

In [31]:
y_test_authors = [y.split("__")[0] for y in y_test]

In [40]:
%%time
nb = MultinomialNB(alpha=0.00001, fit_prior=False)
nb.fit(X_train, y_train_authors)

CPU times: user 1min 18s, sys: 668 ms, total: 1min 19s
Wall time: 1min 19s


In [41]:
%%time
from sklearn.metrics import accuracy_score
preds = nb.predict(X_test)
print(accuracy_score(y_test_authors, preds))

0.60590217461
CPU times: user 1min 18s, sys: 856 ms, total: 1min 19s
Wall time: 1min 19s


In [42]:
probs = nb.predict_proba(X_test[:100])

In [43]:
probs

array([[  4.97274144e-03,   4.90674204e-04,   8.62198136e-06, ...,
          2.45726009e-04,   1.19921565e-02,   2.81849550e-03],
       [  7.36012931e-03,   1.65195559e-04,   8.00881342e-06, ...,
          2.31234700e-04,   1.01864427e-03,   2.11504229e-03],
       [  1.73113690e-12,   4.75216414e-14,   1.53592070e-15, ...,
          3.54175881e-13,   1.05191271e-15,   8.65668152e-11],
       ..., 
       [  1.90269805e-08,   4.50320873e-10,   5.93732120e-12, ...,
          2.75332980e-11,   1.45413891e-10,   9.98293174e-10],
       [  5.15162188e-04,   1.07883640e-03,   2.39489975e-08, ...,
          9.38622484e-04,   9.19465945e-03,   2.29554923e-03],
       [  6.44355436e-04,   3.41505664e-03,   9.04901957e-08, ...,
          2.25966051e-03,   1.61112192e-02,   1.85214379e-02]])

In [44]:
probs[0].shape

(142,)

In [46]:
len(set(y_test_authors))

142

0.14869105294563928

merchant_id=10004979&merchant_key=n8n01mtnbmxol&amount=3945item_name=TestName&subscription_type=1&billing_date=2017-07-25&recurring_amount=3945&frequency=3&cycles=0

In [ ]:
https://sandbox.payfast.co.za/eng/process?merchant_id=10004979&merchant_key=n8n01mtnbmxol&return_url=http%3A//devel.hyperiondev.com/course_payment_success/4362/59f132642a7196d6be5de1363ea6d0487c5e2086f91b9295feb6db77a712e7fb/&cancel_url=http%3A//devel.hyperiondev.com/portal/cpp_prog_essentials/&notify_url=http%3A//devel.hyperiondev.com/course_payment_notify_req/4362/59f132642a7196d6be5de1363ea6d0487c5e2086f91b9295feb6db77a712e7fb/&amount=3945&item_name=MD%20Recurring%20Billing:%20C++%20Programming%20Essentials&item_description=MicroDegree%20Recurring%20Billing%20&subscription_type=1&billing_date=2017-07-25&recurring_amount=3945&frequency=3&cycles=0